In [ ]:
import pandas as pd
import geopandas as gpd
from calitp_data_analysis.gcs_geopandas import GCSGeoPandas
gcsgp = GCSGeoPandas()

In [ ]:
import zipfile

In [ ]:
! pip install pygris

In [ ]:
from _utils import GCS_PATH

In [ ]:
import _utils
import importlib
importlib.reload(_utils)

In [ ]:
import shared_utils
from calitp_data_analysis import geography_utils, utils

# Selecting Corridors

* also map and get per-corridor VMT-ridership results

In [ ]:
analysis_date = '2023-04-15'

In [ ]:
feeds = shared_utils.gtfs_utils_v2.schedule_daily_feed_to_gtfs_dataset_name(selected_date=analysis_date)

In [ ]:
tracts = _utils.get_tract_geoms()

In [ ]:
tracts.GEOID = tracts.GEOID.astype('int64')

In [ ]:
trips_all = gcsgp.read_parquet(f'{GCS_PATH}outputs/new_trips_with_uza.parquet')

In [ ]:
def trips_to_stops(trip_df, feed_list):
    st = shared_utils.gtfs_utils_v2.get_stop_times(analysis_date, feed_list, trip_df=trip_df,
                                                  get_df=True)
    st = st[['stop_id', 'stop_sequence']].drop_duplicates()
    st = stops[['stop_id', 'geometry']].merge(st, on='stop_id')
    return st

In [ ]:
def sjoin_tracts(stops_gdf, tracts_gdf, buffer_m):
    
    stops_gdf = stops_gdf.to_crs(geography_utils.CA_NAD83Albers_m)
    assert stops_gdf.crs == tracts_gdf.crs
    
    stops_gdf.geometry = stops_gdf.buffer(buffer_m)
    tracts_sjoined = gpd.sjoin(tracts_gdf, stops_gdf).drop_duplicates(subset=['GEOID'])
    
    return tracts_sjoined

## Wilshire

In [ ]:
metro = feeds.query("name.str.contains('LA Metro Bus')")
metro

In [ ]:
stops = shared_utils.gtfs_utils_v2.get_stops(analysis_date, metro.feed_key)

In [ ]:
trips = shared_utils.gtfs_utils_v2.get_trips(analysis_date, metro.feed_key)

In [ ]:
trips.route_short_name.unique()

In [ ]:
trips_720 = trips.query('route_short_name.isin(["720"]) & direction_id == 0')

In [ ]:
st_720 = trips_to_stops(trips_720, metro.feed_key)

In [ ]:
#  only keep W of Wil/Wstn
st_720 = st_720.query('stop_sequence <= 11')

In [ ]:
# st_720.explore()

In [ ]:
wilshire = sjoin_tracts(st_720, tracts, 804)

In [ ]:
# wilshire.explore()

In [ ]:
utils.geojson_gcs_export(wilshire, f'{GCS_PATH}intermediate/', 'wilshire')

In [ ]:
# includes non-corridor vmt...
wilshire_results = trips_all.query('GEOID.isin(@wilshire.GEOID)')

In [ ]:
# utils.make_zipped_shapefile(wilshire_results, 'wilsh')

In [ ]:
wilshire_results.explore(column = 'projected_new_transit_trips', scheme = 'NaturalBreaks')

In [ ]:
wilshire_results.sum()

In [ ]:
wilshire_results.describe()

## Fresno Route 1

In [ ]:
fresno = feeds.query("name.str.contains('Fresno Sch')")
fresno

In [ ]:
stops = shared_utils.gtfs_utils_v2.get_stops(analysis_date, fresno.feed_key)

In [ ]:
trips = shared_utils.gtfs_utils_v2.get_trips(analysis_date, fresno.feed_key)

In [ ]:
trips.route_short_name.unique()

In [ ]:
trips_1 = trips.query("route_short_name.isin(['01']) & direction_id == 0 & trip_instance_key == 'db65a5adda0fc0a2744580354516ac68'")

In [ ]:
st_1 = trips_to_stops(trips_1, fresno.feed_key)

In [ ]:
st_1 = st_1.query('stop_sequence < 20') #  vertical portion only

In [ ]:
# st_1.explore()

In [ ]:
fresno = sjoin_tracts(st_1, tracts, 804) #  half-mile

In [ ]:
# fresno.explore()

In [ ]:
utils.geojson_gcs_export(fresno, f'{GCS_PATH}intermediate/', 'fresno')

In [ ]:
# includes non-corridor vmt...
fresno_results = trips_all.query('GEOID.isin(@fresno.GEOID)')

In [ ]:
fresno_results.explore(column = 'projected_new_transit_trips', scheme = 'NaturalBreaks')

In [ ]:
fresno_results.sum()

In [ ]:
fresno_results.describe()

## San Pablo Ave

In [ ]:
ac = feeds.query("name.str.contains('AC Transit')")
ac

In [ ]:
stops = shared_utils.gtfs_utils_v2.get_stops(analysis_date, ac.feed_key)

In [ ]:
trips = shared_utils.gtfs_utils_v2.get_trips(analysis_date, ac.feed_key)

In [ ]:
trips.route_short_name.unique()

In [ ]:
trips_72r = trips.query("route_short_name.isin(['72R']) & direction_id == 0")

In [ ]:
st_72r = trips_to_stops(trips_72r, ac.feed_key)

In [ ]:
# st_72r.explore()

In [ ]:
san_pablo = sjoin_tracts(st_72r, tracts, 804) #  half-mile

In [ ]:
# san_pablo.explore()

In [ ]:
utils.geojson_gcs_export(san_pablo, f'{GCS_PATH}intermediate/', 'san_pablo')

In [ ]:
# includes non-corridor vmt...
san_pablo_results = trips_all.query('GEOID.isin(@san_pablo.GEOID)')

In [ ]:
san_pablo_results.explore(column = 'projected_new_transit_trips', scheme = 'NaturalBreaks')

In [ ]:
san_pablo_results.sum()

In [ ]:
san_pablo_results.describe()

## Eureka H Street/Purple Route

In [ ]:
eureka = feeds.query("name.str.contains('Humboldt Schedule')")
eureka

In [ ]:
stops = shared_utils.gtfs_utils_v2.get_stops(analysis_date, eureka.feed_key)

In [ ]:
trips = shared_utils.gtfs_utils_v2.get_trips(analysis_date, eureka.feed_key)

In [ ]:
trips.route_short_name.unique()

In [ ]:
trips.route_long_name.unique()

In [ ]:
trips_rainbow = trips.query("route_long_name.isin(['Rainbow Route']) & direction_id == 0")

In [ ]:
st_rainbow = trips_to_stops(trips_rainbow, eureka.feed_key)

In [ ]:
st_rainbow = st_rainbow.query('stop_sequence >= 35')

In [ ]:
# st_rainbow.explore()

In [ ]:
eureka = sjoin_tracts(st_rainbow, tracts, 804) #  half-mile

In [ ]:
utils.geojson_gcs_export(eureka, f'{GCS_PATH}intermediate/', 'eureka')

In [ ]:
# includes non-corridor vmt...
eureka_results = trips_all.query('GEOID.isin(@eureka.GEOID)')

In [ ]:
eureka_results.explore(column = 'projected_new_transit_trips', scheme = 'NaturalBreaks')

In [ ]:
eureka_results.sum()

In [ ]:
eureka_results.describe()

# All Corridors Summary

In [ ]:
fresno_results['corridor'] = 'Fresno'
san_pablo_results['corridor'] = 'San Pablo Ave'
wilshire_results['corridor'] = 'Wilshire'
eureka_results['corridor'] = 'Eureka'

In [ ]:
all_results = pd.concat([fresno_results, san_pablo_results, wilshire_results, eureka_results])

In [ ]:
grouped = all_results.groupby('corridor').agg(
    total_new_transit_trips = ('projected_new_transit_trips', 'sum'),
    total_population = ('total_pop', 'sum'),
    total_vmt = ('total_mi_auto', 'sum'),
    p50_auto_trip_mi =('p50_mi_auto', 'median'),
    total_auto_trips = ('total_trips_auto', 'sum')
).reset_index()
grouped

In [ ]:
# grouped.to_csv(f'vmt_transit_corridors.csv')